We have learned to build micrograds and autograds from scratch. We used a scalar class called "Value". However, as a Neural Network grows in size, it is extremely computationally inefficient to work with scalars only. Thus we need to implement a tensor class which holds matrices of nxm dimensions.

In [3]:
import numpy as np

In [148]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    
    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
        

In [149]:
test1 = [[3,5],
        [2,1]]
test2 = [[2,2],
        [2,3]]
t1 = tensor(test1)
t2 = tensor(test2)

t1 @ t2
t1.transpose()

Tensor Values = [[3 2]
 [5 1]]

In [150]:
a = tensor([[1, 2]])
b = tensor([[3, 4]])
c = a + b
d = c * a

print(d._prev)     
print(c._prev)     
print(d._operation) 
print(c._operation) 

{Tensor Values = [[4 6]], Tensor Values = [[1 2]]}
{Tensor Values = [[3 4]], Tensor Values = [[1 2]]}
*
+


Lets save our current state of Tensor class as a checkpoint. Similar to Value class for micrograd and autograd, we have implemented operations and children tracking. We have successfully implemented the computation graph model for tensors. We can now move on to back propagation.

In [153]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        

In [155]:
t = tensor()
print(t.grad)

None


Now let us add the backward lamda functions for each operationn

In [ ]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        self.rows = fromArray.shape[0]
        self.columns = fromArray.shape[1]
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros((rows, columns), dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def random(cls, rows, columns, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.rand(rows, columns)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    @classmethod
    def const(cls, rows, columns, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full((rows, columns), constant)).astype(dtype=dtype)
        t.rows = rows
        t.columns = columns
        return t
    
    
    def shape(self):
        return (self.rows, self.columns)
    
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            


        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix - other.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        out_matrix = other.matrix - self.matrix
        out = tensor(out_matrix, (self, other), '+')
        return out
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        out = tensor(out_matrix, (self, other), '*')
        return out
    
    def __rmul__(self, other):
        return self * other

        
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        out = tensor(out_matrix, (self, other), '@')

        return out
    
    def transpose(self):
        out_matrix = self.matrix.transpose()
        out = tensor(out_matrix, (self, ), 'T')

        return out

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.rows, self.columns, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        assert other.shape() == self.shape(), "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        